### 1. Import and data loading

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Load data
df = pd.read_csv("data/chess_games_raw.csv")
df

/tmp/ipykernel_401946/2651139906.py:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/chess_games_raw.csv")


,Event,Site,Date,Round,White,Black,Result,ECO,WhiteElo,BlackElo,PlyCount,EventDate,EventType
0,Rated Blitz game,lichess.org,2024.01.01,?,Peixeiro,VaRYemezAmca72,1-0,A14,2504,2371,49.0,2024.??.??,blitz
1,Rated Blitz game,lichess.org,2024.01.01,?,bayad2016,Yoda-wins,1/2-1/2,E01,2487,2596,71.0,2024.??.??,blitz
2,Rated Blitz game,lichess.org,2024.01.01,?,FantacticEman,why_this_bot_exists,1/2-1/2,D35,2915,2921,59.0,2024.??.??,blitz
3,Rated Blitz game,lichess.org,2024.01.01,?,Chigorinez,Luhrman,1-0,B40,2542,2433,71.0,2024.??.??,blitz
4,Rated Blitz game,lichess.org,2024.01.01,?,syad_bony77,GERINDRA_BOS,0-1,C48,2473,2658,56.0,2024.??.??,blitz
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269261,Rated Blitz game,?,2024.12.31,-,AfricanJo,Yuri_Guenther,1/2-1/2,C47,2665,2655,NaN,NaN,NaN
3269262,Rated Blitz game,?,2024.12.31,-,cad2024,emiliofelixramirez,1-0,D23,2537,2449,NaN,NaN,NaN
3269263,Rated Blitz game,?,2024.12.31,-,learningchessreally,BiletskiyDanylo,0-1,A15,2583,2557,NaN,NaN,NaN
3269264,Rated Blitz game,?,2024.12.31,-,lotrisking,ezis71,1-0,D30,2513,2398,NaN,NaN,NaN


### 2. Initial data exploration 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3269266 entries, 0 to 3269265
Data columns (total 13 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Event      object 
 1   Site       object 
 2   Date       object 
 3   Round      object 
 4   White      object 
 5   Black      object 
 6   Result     object 
 7   ECO        object 
 8   WhiteElo   int64  
 9   BlackElo   int64  
 10  PlyCount   float64
 11  EventDate  object 
 12  EventType  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 324.3+ MB


In [4]:
df.isnull().sum()

Event              0
Site               0
Date               0
Round              0
White              0
Black              0
Result             0
ECO              336
WhiteElo           0
BlackElo           0
PlyCount     2707355
EventDate    2707355
EventType    2711070
dtype: int64

In [5]:
df.describe()

,WhiteElo,BlackElo,PlyCount
count,3.269266e+06,3.269266e+06,561911.000000
mean,2.569311e+03,2.568953e+03,87.919028
std,1.342731e+02,1.328460e+02,40.879115
min,2.300000e+03,2.300000e+03,0.000000
25%,2.500000e+03,2.500000e+03,60.000000
50%,2.545000e+03,2.545000e+03,81.000000
75%,2.620000e+03,2.620000e+03,110.000000
max,3.367000e+03,3.390000e+03,600.000000


In [6]:
df['Date'].value_counts()

Date
2024.12.28    13967
2024.05.11    11704
2024.02.10    11302
2024.01.22    10984
2024.01.13    10935
              ...  
2024.06.22     7495
2024.08.24     7342
2024.08.03     7209
2024.10.26     7143
2024.09.12     4500
Name: count, Length: 366, dtype: int64

### 3. Drop columns that has many missing data or irrelevant

In [7]:
df = df.drop(['Site', 'Round', 'PlyCount','EventDate','EventType'], axis=1)
df = df.dropna(subset=['ECO'])
df.isnull().sum()

Event       0
Date        0
White       0
Black       0
Result      0
ECO         0
WhiteElo    0
BlackElo    0
dtype: int64

### 4. Convert data types

In [8]:
df['WhiteElo'] = df['WhiteElo'].astype(int)
df['BlackElo'] = df['BlackElo'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3268930 entries, 0 to 3269265
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Event     object
 1   Date      object
 2   White     object
 3   Black     object
 4   Result    object
 5   ECO       object
 6   WhiteElo  int64 
 7   BlackElo  int64 
dtypes: int64(2), object(6)
memory usage: 224.5+ MB


### 5. Remove * or unfinished games rows 

In [9]:
df = df[df['Result'].isin(['1-0', '0-1', '1/2-1/2'])]
df['Result'].value_counts()

Result
1-0        1508107
0-1        1338945
1/2-1/2     421755
Name: count, dtype: int64

### 6. Drop duplicates 

In [10]:
df[df.duplicated()]

,Event,Date,White,Black,Result,ECO,WhiteElo,BlackElo
821,Rated Blitz game,2024.01.01,TorBot_v2,Boller1910,1/2-1/2,C43,2927,2919
881,Rated Rapid game,2024.01.01,Intron014,YoBot_v2,1/2-1/2,D02,2938,2931
2561,Rated Blitz game,2024.01.01,Boller1910,Fruity23,1/2-1/2,C67,2924,2926
3002,Rated Blitz game,2024.01.01,TorBot_v2,Blauer-Rausch,1/2-1/2,C67,2927,2930
5031,Rated Blitz game,2024.01.01,ReinforcementTrial,torbot_v2,1/2-1/2,A28,2926,2927
...,...,...,...,...,...,...,...,...
3267206,Rated Blitz game,2024.12.31,VanuatuLover,YOUPLAYCHESS,1-0,A03,2773,2779
3267300,Rated Blitz game,2024.12.31,VanuatuLover,YOUPLAYCHESS,0-1,A03,2784,2767
3267351,Rated Blitz game,2024.12.31,YOUPLAYCHESS,VanuatuLover,1-0,B10,2773,2778
3268962,Rated Blitz game,2024.12.31,imnotsohuman,PetersBot,1/2-1/2,C89,3010,3033


In [11]:
df = df.drop_duplicates()

### 7. Parse and group game type into 3 categories

In [12]:
#df['Event'].unique()
df['Event'].value_counts()

Event
Rated Blitz game                                                  2884607
Rated Rapid game                                                   186923
Rated Classical game                                                15822
Rated Blitz tournament https://lichess.org/tournament/winter24       5587
Rated Blitz tournament https://lichess.org/tournament/spring24       3705
                                                                   ...   
Rated Blitz tournament https://lichess.org/tournament/7YTm7KQy          1
Rated Rapid tournament https://lichess.org/tournament/m0HqiSX2          1
Rated Blitz tournament https://lichess.org/tournament/uHrqxY12          1
Rated Blitz tournament https://lichess.org/tournament/H5Sx6aJK          1
Rated Blitz tournament zzMAjwud                                         1
Name: count, Length: 8881, dtype: int64

In [13]:
def extract_time_class(event):
    event_lower = str(event).lower()
    if 'blitz' in event_lower:
        return 'blitz'
    elif 'rapid' in event_lower:
        return 'rapid'
    elif 'classical' in event_lower:
        return 'classical'
    else:
        return np.nan

df['EventType'] = df['Event'].apply(extract_time_class)
df = df[df['EventType'].notna()]
df['EventType'].value_counts(dropna=False)

EventType
blitz        3059114
rapid         191149
classical      15869
Name: count, dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3266132 entries, 0 to 3269265
Data columns (total 9 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   Event      object
 1   Date       object
 2   White      object
 3   Black      object
 4   Result     object
 5   ECO        object
 6   WhiteElo   int64 
 7   BlackElo   int64 
 8   EventType  object
dtypes: int64(2), object(7)
memory usage: 249.2+ MB


### 8. Export cleaned data

In [15]:
df.to_csv('data/chess_games_clean.csv', index=False)